### Extract the logatirhm of a SU(2) gauge link

Input the list of coefficients for a SU(2) gauge link parametrization, see Eq. (C.7) from [David Mueller's PhD Thesis](https://arxiv.org/abs/1904.04267), more concisely $(u_0, u_1, u_2, u_3)$. This gauge link is obtained from a Glasma SU(2) simulation, using Pooja's parameters.

In [1]:
ux_param = (0.99910964, -0.02292065, 0.03035651, 0.01824954)

### Import packages

In [2]:
import numpy as np
import math

#### Input the Pauli matrices and the 2x2 identity matrix as tuples

In [3]:
def complex_tuple(*t):
    return tuple(map(np.complex64, t))

# Pauli matrices
id0 = complex_tuple(1, 0, 0, 1)
s1 = complex_tuple(0, 1, 1, 0)
s2 = complex_tuple(0, -1j, 1j, 0)
s3 = complex_tuple(1, 0, 0, -1)

slist = (id0, s1, s2, s3)

Convert from SU(2) parametrization coefficients $(u_0, u_1, u_2, u_3)$ to a SU(2) matrix as $\boxed{U=u_0\mathbb{1}+\sum_a u_a\sigma_a/2}$. Notice that all SU(2) operations (`add`, `mul`, `dagger` and others) are done element-wise. This is slow on the CPU but becomes really fast on GPU.

In [4]:
def get_algebra_matrix(algebra_param_coeff):
    """
    Contruct the matrix U=u_0 id + i sum_a u_a \sigma_a/2
    """
    ms0 = mul_s(s1, algebra_param_coeff[1])
    ms1 = mul_s(s2, algebra_param_coeff[2])
    ms2 = mul_s(s3, algebra_param_coeff[3])

    # Add all
    b0 = add(ms0, ms1)
    b1 = add(b0, ms2)

    # Overall factor
    res1 = mul_s(b1, 0.5j)

    ms3 = mul_s(id0, algebra_param_coeff[0])
    res = add(res1, ms3)
    return res

# group add: g0 = g0 + f * g1
def add(g0, g1):
    r0 = g0[0] + g1[0]
    r1 = g0[1] + g1[1]
    r2 = g0[2] + g1[2]
    r3 = g0[3] + g1[3]
    return r0, r1, r2, r3

# multiply by scalar
def mul_s(g0, f):
    r0 = np.complex64(f) * g0[0]
    r1 = np.complex64(f) * g0[1]
    r2 = np.complex64(f) * g0[2]
    r3 = np.complex64(f) * g0[3]
    return r0, r1, r2, r3


Extract the SU(2) matrix, see that it's close to identity, with small diagonal elements and imaginary parts

In [5]:
ux = get_algebra_matrix(ux_param)
np.array(ux)

array([ 0.9991096 +0.00912477j,  0.01517825-0.01146032j,
       -0.01517825-0.01146032j,  0.9991096 -0.00912477j], dtype=complex64)

Check that the resulting matrix is unitary $U U^\dagger=\mathbb{1}$ and its determinant is unity $\mathrm{det}U=1$

In [6]:
def mul(a, b):
    """SU(2) multiplication: 2x2 matrix multiplication

    >>> a=[1, 2, 3, 4]
    >>> b=[5, 6, 7, 8]
    >>> c=mul(a,b)
    >>> c
    (19, 22, 43, 50)

    # Check with numpy
    >>> import numpy as np
    >>> ma = np.asarray(a).reshape(2,2)
    >>> mb = np.asarray(b).reshape(2,2)
    >>> mc = np.matmul(ma, mb)
    >>> mc
    array([[19, 22],
           [43, 50]])

    >>> tuple(mc.flatten()) == c
    True
    """
    r0 = a[0] * b[0] + a[1] * b[2] 
    r1 = a[0] * b[1] + a[1] * b[3] 
    r2 = a[2] * b[0] + a[3] * b[2] 
    r3 = a[2] * b[1] + a[3] * b[3] 
    return r0, r1, r2, r3

# conjugate transpose
def dagger(a):
    r0 = a[0].conjugate()
    r1 = a[2].conjugate()
    r2 = a[1].conjugate()
    r3 = a[3].conjugate()
    return r0, r1, r2, r3

# trace ( a . dagger(a) )
def sq(a): 
    """
    >>> a = (1,2,3,4)
    >>> ta = sq(a)
    >>> ta
    30.0

    >>> tma = tr(mul(a, dagger(a))).real
    >>> tma
    30

    >>> tma == ta
    True

    :param a:
    :return:
    """
    # return tr(mul(a, dagger(a))).real

    s = np.float64(0)
    for i in range(4):
        s += a[i].real * a[i].real + a[i].imag * a[i].imag
    return s

def check_unitary(u):  
    x = mul(u, dagger(u))
    d = add(x, mul_s(id0, -1))
    s = sq(d)
    if s > 1e-4:
       print("Unitarity is violated")  
    else:
        print("Unitarity is satisfied")
    return s

# determinant
def det(a):
    res  = a[0] * a[3] - a[1] * a[2]
    return res

In [7]:
check_unitary(ux)

Unitarity is satisfied


3.564264261513017e-06

In [8]:
print("The determinant is", det(ux))

The determinant is (0.99866503+0j)


- - -
#### Extract the logarithm of a SU(2) gauge link
The function `mlog(a)` evaluates the Taylor series expansion of the logarithm of a matrix `a`. The result is compared with `logm` from the `linalg` library of [SciPy](https://scipy.org/), more details [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.logm.html).

In [9]:
LOG_MIN_TERMS = -1 # minimum number of terms in Taylor series
LOG_MAX_TERMS = 100 # maximum number of terms in Taylor series
LOG_ACCURACY_SQUARED = 1.e-32 # 1.e-32 # accuracy

# logarithm map
def mlog(a):
    """
    Computes logarithm of a matrix using Taylor series

    mlog(a) = (a-id0) - (a-id0)^2 / 2 + (a-id0)^3 / 3 - (a-id0)^4 / 4 + ...

    Works for matrices close to identity (for example gauge links)
    Example and comparison with logm() from Scipy:

    >>> Ux = [0.98510928-0.15514118j, 0.06072216+0.04247033j, -0.05816349+0.04591208j, 0.99243458+0.09789123j]
    >>> lnUx_CUDA = mlog(Ux)
    >>> lnUx_CUDA
    ((-5.283685708374216e-09-0.1559693702453399j), (0.05968301710568872+0.04436976704847219j), (-0.059683020380762314+0.044369774147350945j), (-6.818738708510321e-10+0.0980854807916348j))
    >>> from scipy.linalg import logm
    >>> lnUx_Scipy = np.reshape(logm(np.reshape(Ux, (2, 2))), 2*2)
    >>> lnUx_Scipy
    array([-5.28368542e-09-0.15596937j,  5.96830171e-02+0.04436977j,
           -5.96830204e-02+0.04436977j, -6.81873818e-10+0.09808548j])
    >>> lnUx_CUDA - lnUx_Scipy
    array([-2.91666427e-16+0.00000000e+00j, -5.55111512e-17-6.93889390e-17j,
           4.85722573e-17+4.85722573e-17j, -5.30250034e-17-9.71445147e-17j])
    """
    res = add(a, mul_s(id0, -1))
    t = add(a, mul_s(id0, -1))
    sign = -1
    for i in range(1, LOG_MAX_TERMS):
        t = mul(t, add(a, mul_s(id0, -1)))
        buff = mul_s(t, sign/(i+1))
        res = add(res, buff)
        sign = sign * (-1)
        n = sq(t) 
        if (i > LOG_MIN_TERMS) and (math.fabs(n.real) < LOG_ACCURACY_SQUARED):
            break
        # else:
            # print("Logarithm did not reach desired accuracy") 

    return res

In [10]:
lnux_taylor = mlog(ux)
np.array(lnux_taylor)

array([-0.00066793+0.00913155j,  0.01518953-0.01146883j,
       -0.01518953-0.01146883j, -0.00066793-0.00913155j], dtype=complex64)

In [11]:
from scipy.linalg import logm
lnux_scipy = np.reshape(logm(np.reshape(ux, (2, 2))), 2*2)
lnux_scipy

logm result may be inaccurate, approximate err = 1.7764702346012432e-07


array([-0.00066804+0.00913154j,  0.01518955-0.01146884j,
       -0.01518947-0.01146888j, -0.00066794-0.00913151j])

Comparison of the two ways to extract the logarithm. The difference is of the order of $10^{-7}$, thus they agree very well

In [12]:
lnux_taylor - lnux_scipy

array([ 1.09600173e-07+5.26611089e-09j, -2.89470559e-08+6.23388672e-09j,
       -5.19192634e-08+4.98090421e-08j,  9.62030791e-09-3.45076762e-08j])

---
### Simulation parameters
Chosen to match Pooja's, namely
- Gauge group SU(3)
- Nb. of color sheets $N_s=1$
- Nb. of lattice points $N=512$
- Transverse simulation length $L=2\,\mathrm{fm}$
- Leapfrog time step $1/8=0.125$
- Simulation time $\tau_\mathrm{stop}=0.4\,\mathrm{fm/}c$
- Saturation momentum $Q_s=3\,\mathrm{GeV}$
- Coupling constant $g=\sqrt{2\pi\alpha_s}$ with runnning coupling $\alpha_s=\dfrac{2\pi}{\dfrac{33-2 N_f}{3}}\dfrac{1}{\mathrm{log}\left(\dfrac{Q_s}{\lambda_{\mathrm{QCD}}}\right)}$
- Factor relating the MV model $g^2\mu$ to the saturation scale $Q_s=0.6 g^2\mu$ 

In [5]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 512    
tau_sim = 0.1     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 3        
ns = 1    
factor = 0.6        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Set environment variables
Work with the Numba version, which runs of the CPU.

In [6]:
import os
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
Nc = su.NC

### Simulation routine for evolving the Glasma fields

In [7]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(ev): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Gauge link along x in a lattice point, at the last simulation time step
    lat = 200

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)
            ui_all = s.u1.copy()
            if t==(maxt-1):
                ux = ui_all[lat, 1, :].reshape(Nc,Nc)

            pbar.set_description("Event " + str(ev+1))
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    print(ux)

In [8]:
for ev in range(nevents):
    output = simulate(ev)

Event 1: 100%|██████████| 204/204 [00:47<00:00,  4.26it/s]

[[ 0.9992373 +0.00597116j -0.00113904-0.00153799j -0.02812345+0.02635478j]
 [ 0.00200096-0.00037265j  0.99922377+0.01439965j  0.03599992+0.00666049j]
 [ 0.02845036+0.02599231j -0.03597915+0.00680715j  0.99837853-0.02035784j]]


- - -
### Extract the logatirhm of a SU(3) gauge link

In [1]:
import numpy as np
import math

In [2]:
def complex_tuple(*t):
    return tuple(map(np.complex128, t))

Nc = 3

# Gell-Mann matrices
id0 = complex_tuple(1, 0, 0, 0, 1, 0, 0, 0, 1)
s1 = complex_tuple(0, 1, 0, 1, 0, 0, 0, 0, 0)
s2 = complex_tuple(0, -1j, 0, 1j, 0, 0, 0, 0, 0)
s3 = complex_tuple(1, 0, 0, 0, -1, 0, 0, 0, 0)
s4 = complex_tuple(0, 0, 1, 0, 0, 0, 1, 0, 0)
s5 = complex_tuple(0, 0, -1j, 0, 0, 0, 1j, 0, 0)
s6 = complex_tuple(0, 0, 0, 0, 0, 1, 0, 1, 0)
s7 = complex_tuple(0, 0, 0, 0, 0, -1j, 0, 1j, 0)
s8 = complex_tuple(1 / math.sqrt(3), 0, 0, 0, 1 / math.sqrt(3), 0, 0, 0, -2 / math.sqrt(3))

slist = (id0, s1, s2, s3, s4, s5, s6, s7, s8)

In [3]:
"""
    SU(3) group & algebra functions
"""

# SU(3) group elements are given by 3x3 complex matrices:
#   a[0] a[1] a[2]
#   a[3] a[4] a[5]
#   a[6] a[7] a[8]
# (layout corresponds to C-order of numpy array)

# su3 multiplication

# Generate the matrix multiplication code:
# print("\n".join(["r{} = ".format(j) + " + ".join(["a[{}] * b[{}]".format(3 * (j // 3) + i, 3 * i + (j % 3)) for i in range(3)]) for j in range(9)]))

def mul(a, b):
    """SU(3) multiplication: 3x3 matrix multiplication

    >>> a=[1,2,3,4,5,6,7,8,9]
    >>> b=[3,6,8,4,3,2,1,3,4]
    >>> c=mul(a,b)
    >>> c
    (14, 21, 24, 38, 57, 66, 62, 93, 108)

    # Check with numpy
    >>> import numpy as np
    >>> ma = np.asarray(a).reshape(3,3)
    >>> mb = np.asarray(b).reshape(3,3)
    >>> mc = np.matmul(ma, mb)
    >>> mc
    array([[ 14,  21,  24],
           [ 38,  57,  66],
           [ 62,  93, 108]])

    >>> tuple(mc.flatten()) == c
    True
    """
    r0 = a[0] * b[0] + a[1] * b[3] + a[2] * b[6]
    r1 = a[0] * b[1] + a[1] * b[4] + a[2] * b[7]
    r2 = a[0] * b[2] + a[1] * b[5] + a[2] * b[8]
    r3 = a[3] * b[0] + a[4] * b[3] + a[5] * b[6]
    r4 = a[3] * b[1] + a[4] * b[4] + a[5] * b[7]
    r5 = a[3] * b[2] + a[4] * b[5] + a[5] * b[8]
    r6 = a[6] * b[0] + a[7] * b[3] + a[8] * b[6]
    r7 = a[6] * b[1] + a[7] * b[4] + a[8] * b[7]
    r8 = a[6] * b[2] + a[7] * b[5] + a[8] * b[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

EXP_MIN_TERMS = -1 # minimum number of terms in Taylor series
EXP_MAX_TERMS = 100 # maximum number of terms in Taylor series
EXP_ACCURACY_SQUARED = 1.e-40 # 1.e-32 # accuracy

# exponential map
def mexp(a):
    """ Calculate exponential using Taylor series

    mexp(a) = 1 + a + a^2 / 2 + a^3 / 6 + ...

    >>> a = id0
    >>> mexp(a)
    ((2.7182818284590455+0j), 0j, 0j, 0j, (2.7182818284590455+0j), 0j, 0j, 0j, (2.7182818284590455+0j))

    """
    res = id0
    t = id0
    for i in range(1, EXP_MAX_TERMS):
        t = mul(t, a)
        t = mul_s(t, 1/i)
        res = add(res, t)
        n = sq(t)  # TODO: Is it possible to improve performance by checking this not so often?
        if (i > EXP_MIN_TERMS) and (math.fabs(n.real) < EXP_ACCURACY_SQUARED):
            break
    else:
        # print("Exponential did not reach desired accuracy: {}".format(a))   # TODO: remove debugging code
        print("Exponential did not reach desired accuracy")  # TODO: remove debugging code
    return res

LOG_MIN_TERMS = -1 # minimum number of terms in Taylor series
LOG_MAX_TERMS = 100 # maximum number of terms in Taylor series
LOG_ACCURACY_SQUARED = 1.e-32 # 1.e-32 # accuracy

# logarithm map
def mlog(a):
    """
    Computes logarithm of a matrix using Taylor series

    mlog(a) = (a-id0) - (a-id0)^2 / 2 + (a-id0)^3 / 3 - (a-id0)^4 / 4 + ...

    Works for matrices close to identity (for example gauge links)
    """
    res = add(a, mul_s(id0, -1))
    t = add(a, mul_s(id0, -1))
    sign = -1
    for i in range(1, LOG_MAX_TERMS):
        t = mul(t, add(a, mul_s(id0, -1)))
        buff = mul_s(t, sign/(i+1))
        res = add(res, buff)
        sign = sign * (-1)
        n = sq(t) 
        if (i > LOG_MIN_TERMS) and (math.fabs(n.real) < LOG_ACCURACY_SQUARED):
            break
        # else:
            # print("Logarithm did not reach desired accuracy") 

    return res

# determinant
def det(a):
    res  = a[0] * (a[4] * a[8] - a[5] * a[7])
    res += a[1] * (a[5] * a[6] - a[3] * a[8])
    res += a[2] * (a[3] * a[7] - a[4] * a[6])
    return res

# group add: g0 = g0 + f * g1
def add(g0, g1):
    # Unfortunately, tuple creation from list comprehension does not work in numba:
    # see https://github.com/numba/numba/issues/2771
    #
    # result = tuple(g0[i] + g1[i] for i in range(9))
    # return result
    r0 = g0[0] + g1[0]
    r1 = g0[1] + g1[1]
    r2 = g0[2] + g1[2]
    r3 = g0[3] + g1[3]
    r4 = g0[4] + g1[4]
    r5 = g0[5] + g1[5]
    r6 = g0[6] + g1[6]
    r7 = g0[7] + g1[7]
    r8 = g0[8] + g1[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# multiply by scalar
def mul_s(g0, f):
    # Unfortunately, tuple creation from list comprehension does not work in numba:
    # see https://github.com/numba/numba/issues/2771
    #
    # result = tuple(f * g0[i] for i in range(9))
    # return result
    r0 = np.complex128(f) * g0[0]
    r1 = np.complex128(f) * g0[1]
    r2 = np.complex128(f) * g0[2]
    r3 = np.complex128(f) * g0[3]
    r4 = np.complex128(f) * g0[4]
    r5 = np.complex128(f) * g0[5]
    r6 = np.complex128(f) * g0[6]
    r7 = np.complex128(f) * g0[7]
    r8 = np.complex128(f) * g0[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# conjugate transpose
def dagger(a):
    r0 = a[0].conjugate()
    r1 = a[3].conjugate()
    r2 = a[6].conjugate()
    r3 = a[1].conjugate()
    r4 = a[4].conjugate()
    r5 = a[7].conjugate()
    r6 = a[2].conjugate()
    r7 = a[5].conjugate()
    r8 = a[8].conjugate()
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# trace
def tr(a):
    return a[0] + a[4] + a[8]

# trace ( a . dagger(a) )
def sq(a): # TODO: rename to tr_sq? or tr_abs_sq?
    """
    >>> a = (1,2,3,4,6,8,9,5,4)
    >>> ta = sq(a)
    >>> ta
    252

    >>> tma = tr(mul(a, dagger(a))).real
    >>> tma
    252

    >>> tma == ta
    True

    :param a:
    :return:
    """
    # return tr(mul(a, dagger(a))).real

    s = np.float64(0)
    for i in range(9):
        s += a[i].real * a[i].real + a[i].imag * a[i].imag
    return s

def check_unitary(u):  # TODO: remove debugging code
    x = mul(u, dagger(u))
    d = add(x, mul_s(id0, -1))
    s = sq(d)
    # if s > 1e-8:
    #    print("Unitarity violated")  # TODO: remove debugging code
    return s

In [4]:
# curraun
# ux = complex_tuple(0.9992373 +0.00597116j, -0.00113904-0.00153799j, -0.02812345+0.02635478j, 
#     0.00200096-0.00037265j,  0.99922377+0.01439965j,  0.03599992+0.00666049j, 
#     0.02845036+0.02599231j, -0.03597915+0.00680715j,  0.99837853-0.02035784j)
# pooja
ux = complex_tuple(0.999773332297265+1.200901570480802E-002j, 9.337536969798815E-003+9.373516471449303E-003j,-1.157649128313104E-002+-3.300514581480528E-006j,
    -9.049799378893927E-003+9.456078543778054E-003j, 0.999470695182973+-2.638751157253443E-002j, 1.305022491015412E-002+-4.517083466599165E-003j,
    1.173464879673716E-002+2.220279658133394E-004j, -1.299671643839491E-002+-4.249900891929569E-003j, 0.999734175041685+1.438233914040814E-002j)

In [5]:
check_unitary(ux)

1.4823476966401162e-31

In [6]:
print("The determinant is", det(ux))

The determinant is (1.0000000000000002-3.470743971154097e-18j)


In [7]:
lnux_taylor = np.array(mlog(ux))
lnux_taylor

array([ 1.92188480e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
       -1.16575554e-02+0.00010938j, -9.19523417e-03+0.0094164j ,
        1.08408557e-16-0.02639329j,  1.30256890e-02-0.00438424j,
        1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
       -4.53993708e-17+0.01438351j])

In [14]:
lnux_uudag = np.array(mul_s(add(ux, mul_s(dagger(ux), -1)), 0.5))
lnux_uudag

array([ 0.        +0.01200902j,  0.00919367+0.0094148j ,
       -0.01165557+0.00010936j, -0.00919367+0.0094148j ,
        0.        -0.02638751j,  0.01302347-0.00438349j,
        0.01165557+0.00010936j, -0.01302347-0.00438349j,
        0.        +0.01438234j])

In [9]:
from scipy.linalg import logm
lnux_scipy = np.reshape(logm(np.reshape(ux, (Nc, Nc))), Nc*Nc)
lnux_scipy

array([-2.18141477e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
       -1.16575554e-02+0.00010938j, -9.19523417e-03+0.0094164j ,
       -3.41740525e-16-0.02639329j,  1.30256890e-02-0.00438424j,
        1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
       -1.73472348e-16+0.01438351j])

In [10]:
lnux_taylor - lnux_scipy

array([ 4.10329958e-16-1.04083409e-17j,  2.02962647e-16+3.62557206e-16j,
        1.26634814e-16-5.69409428e-16j,  2.22044605e-16-5.39499001e-16j,
        4.50149081e-16+1.80411242e-16j,  1.21430643e-17+1.26634814e-16j,
       -9.02056208e-17+5.02663232e-16j, -4.51028104e-17-3.20923843e-16j,
        1.28072977e-16-4.85722573e-17j])

In [15]:
lnux_uudag - lnux_scipy

array([ 2.18141477e-16-7.64224652e-07j, -1.56599257e-06-1.60364535e-06j,
        1.98535885e-06-1.86182703e-08j,  1.56599257e-06-1.60364535e-06j,
        3.41740525e-16+5.77598560e-06j, -2.21832781e-06+7.46662893e-07j,
       -1.98535885e-06-1.86182693e-08j,  2.21832781e-06+7.46662893e-07j,
        1.73472348e-16-1.16848827e-06j])

In [12]:
%load_ext fortranmagic

In [16]:
%%fortran --extra "-DF2PY_REPORT_ON_ARRAY_COPY=1"
subroutine mlog_taylor_pooja(input, output)
   complex(8), dimension(3, 3), intent(in) :: input
   complex(8), dimension(3, 3), intent(out) :: output

   complex(8), parameter :: i = (0.0D0, 1.0D0)	
   complex(8), dimension(3, 3) :: II, mat_U, mat_U_I, mat_U_I_pow, log_U
   integer :: j5

   mat_U = input

   II = 0.0D0	
   II(1,1) = (1.0D0, 0.0D0) ;	II(2,2) = (1.0D0, 0.0D0) ;	II(3,3) = (1.0D0, 0.0D0)

   mat_U_I      =  mat_U - II		
   mat_U_I_pow  =  II				
   log_U = (0.0D0, 0.0D0)	

   DO j5 = 1, 100
      mat_U_I_pow  =  MATMUL(mat_U_I, mat_U_I_pow)
      log_U        =  log_U + (((-1.0D0)**(j5+1) )/DBLE(j5))*mat_U_I_pow				
   ENDDO

   output = log_U

end subroutine

In [18]:
ux_array = np.reshape(np.array(ux), (Nc, Nc))
lnux_taylor_pooja = mlog_taylor_pooja(ux_array)
lnux_taylor_pooja

array([[ 1.92188190e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
        -1.16575554e-02+0.00010938j],
       [-9.19523417e-03+0.0094164j ,  1.08407143e-16-0.02639329j,
         1.30256890e-02-0.00438424j],
       [ 1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
        -4.53995715e-17+0.01438351j]])

In [19]:
lnux_taylor - lnux_taylor

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j])

In [20]:
%%fortran --extra "-DF2PY_REPORT_ON_ARRAY_COPY=1"
subroutine mlog_uudag_pooja(input, output)
   complex(8), dimension(3, 3), intent(in) :: input
   complex(8), dimension(3, 3), intent(out) :: output

   complex(8), parameter :: i = (0.0D0, 1.0D0)	
   complex(8), dimension(3, 3) :: mat_U, mat_U_dag, log_U
   integer :: j5

   mat_U = input

   mat_U_dag  = DCONJG(TRANSPOSE(mat_U))				
   log_U =	(1/2.0D0)*(mat_U - mat_U_dag)

   output = log_U

end subroutine

In [21]:
lnux_uudag_pooja = mlog_uudag_pooja(ux_array)
lnux_uudag_pooja

array([[ 0.        +0.01200902j,  0.00919367+0.0094148j ,
        -0.01165557+0.00010936j],
       [-0.00919367+0.0094148j ,  0.        -0.02638751j,
         0.01302347-0.00438349j],
       [ 0.01165557+0.00010936j, -0.01302347-0.00438349j,
         0.        +0.01438234j]])

In [22]:
lnux_uudag - lnux_uudag

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j])